# Some code
This file converts World Bank netcdf data into csv format.

## Prepare
install the following packages if not installed.

In [13]:
# %pip install pandas
# %pip install xarray
# %pip install ttictoc

Set working directory.

In [2]:
import pandas as pd
import xarray as xr
import os

In [3]:
print("The Current working directory is: {0}".format(os.getcwd()))

The Current working directory is: C:\Users\user\Documents


In [4]:
os.chdir(r"C:\Users\user\Desktop\RA file\EnergyAndWeather\wb_data") # change here
print("Working directory has been changed to: {0}".format(os.getcwd()))

Working directory has been changed to: C:\Users\user\Desktop\RA file\EnergyAndWeather\wb_data


In [5]:
# netcdf files are stored in this directory.
%ls

 Volume in drive C is system
 Volume Serial Number is 4C26-1A36

 Directory of C:\Users\user\Desktop\RA file\EnergyAndWeather\wb_data

08/12/2023  12:08 PM    <DIR>          .
08/12/2023  12:04 PM    <DIR>          ..
08/11/2023  08:18 PM       625,719,410 final_wb_data.dta
08/10/2023  11:26 PM        74,720,056 five_day_rainfall.nc
08/10/2023  11:59 PM        74,719,805 frost_day.nc
08/11/2023  08:32 PM            23,859 gen_csv_wb_data.ipynb
08/11/2023  12:24 AM        74,720,998 hi35_day.nc
08/11/2023  01:11 AM        74,719,805 hot_day.nc
08/11/2023  01:11 AM        74,719,805 ice_day.nc
08/10/2023  10:11 PM        74,720,045 max_dry_day.nc
08/10/2023  10:20 PM        74,720,045 max_wet_day.nc
08/11/2023  08:37 PM             3,277 merge_wb_data.do
08/10/2023  11:12 PM        74,720,018 one_day_rainfall.nc
08/12/2023  12:08 PM        74,719,989 rainfall.nc
08/11/2023  10:54 PM    <DIR>          temp_csv_dta_files
08/12/2023  12:07 PM        74,719,869 temper.nc
08/10/2023  11:48 PM

The csv file generated will be big. We define a function to report file size later.

In [7]:
from pathlib import Path
 
 
def get_file_size(file):
    stat = os.stat(file)
    size = stat.st_size
    return size

 
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('csv size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('csv size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    elif unit == "GB":
        return print('csv size: ' + str(round(size / (1024 * 1024 * 1024), 3)) + ' Gigabytes')
    else:
        return print('csv size: ' + str(size) + ' bytes')

Import time to report time elapsed.

In [6]:
import time
from ttictoc import tic,toc

## Convert netcdf into csv format
Use `max_dry_day.nc` as an example.

In [5]:
filename="max_dry_day.nc"
ds = xr.open_dataset(filename)
print(ds)

<xarray.Dataset>
Dimensions:                     (year: 71, lat: 360, lon: 720, bnds: 2)
Coordinates:
  * year                        (year) float64 1.95e+03 1.951e+03 ... 2.02e+03
  * lat                         (lat) float64 -89.75 -89.25 ... 89.25 89.75
  * lon                         (lon) float64 -179.8 -179.2 ... 179.2 179.8
  * bnds                        (bnds) int32 0 1
Data variables:
    timeseries-cdd-annual-mean  (year, lat, lon) float32 ...
    lon_bnds                    (lon, bnds) float64 ...
    lat_bnds                    (lat, bnds) float64 ...
Attributes:
    comment:              Processed ERA5 data by WB; RAW-data from ECMWF: htt...
    NCO:                  netCDF Operators version 4.7.9 (Homepage = http://n...
    history:              Fri Nov 13 19:17:20 2020: ncks -4 --ppc default=7 e...
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    Conventions:          CF-1.6
    CONVERSION_DATE:      Fri Nov 13 19:17:11 MST 2020
    C

In [6]:
df = ds.to_dataframe()
print(df)

                            timeseries-cdd-annual-mean  lon_bnds  lat_bnds
year   lat    lon     bnds                                                
1950.0 -89.75 -179.75 0                           87.0    -180.0     -90.0
                      1                           87.0    -179.5     -89.5
              -179.25 0                           87.0    -179.5     -90.0
                      1                           87.0    -179.0     -89.5
              -178.75 0                           87.0    -179.0     -90.0
...                                                ...       ...       ...
2020.0  89.75  178.75 1                           30.0     179.0      90.0
               179.25 0                           30.0     179.0      89.5
                      1                           30.0     179.5      90.0
               179.75 0                           30.0     179.5      89.5
                      1                           30.0     180.0      90.0

[36806400 rows x 3 colum

In [7]:
# rename key variables
df = df.rename(columns = {'timeseries-cdd-annual-mean':filename[:-3]}, inplace = False)
print(df)

                            max_dry_day  lon_bnds  lat_bnds
year   lat    lon     bnds                                 
1950.0 -89.75 -179.75 0            87.0    -180.0     -90.0
                      1            87.0    -179.5     -89.5
              -179.25 0            87.0    -179.5     -90.0
                      1            87.0    -179.0     -89.5
              -178.75 0            87.0    -179.0     -90.0
...                                 ...       ...       ...
2020.0  89.75  178.75 1            30.0     179.0      90.0
               179.25 0            30.0     179.0      89.5
                      1            30.0     179.5      90.0
               179.75 0            30.0     179.5      89.5
                      1            30.0     180.0      90.0

[36806400 rows x 3 columns]


In [16]:
# save as csv
tic()

df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')


elapsed = toc()
print('Elapsed time:',elapsed)

Elapsed time: 199.33929199998965


In [17]:
# report csv file size
csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "GB")

csv size: 1.355 Gigabytes


## Drop unnecessary variables 

After we delete (`bnds`,`lon_buds`,`lat_bnds`) and drop duplicates from the csv file, the file size will be smaller.

In [18]:
import pandas as pd

tic()

df = pd.read_csv(filename[:-3]+'.csv')
print(df)

print('Elapsed time:',toc())

            year    lat     lon  bnds  max_dry_day  lon_bnds  lat_bnds
0         1950.0 -89.75 -179.75     0         87.0    -180.0     -90.0
1         1950.0 -89.75 -179.75     1         87.0    -179.5     -89.5
2         1950.0 -89.75 -179.25     0         87.0    -179.5     -90.0
3         1950.0 -89.75 -179.25     1         87.0    -179.0     -89.5
4         1950.0 -89.75 -178.75     0         87.0    -179.0     -90.0
...          ...    ...     ...   ...          ...       ...       ...
36806395  2020.0  89.75  178.75     1         30.0     179.0      90.0
36806396  2020.0  89.75  179.25     0         30.0     179.0      89.5
36806397  2020.0  89.75  179.25     1         30.0     179.5      90.0
36806398  2020.0  89.75  179.75     0         30.0     179.5      89.5
36806399  2020.0  89.75  179.75     1         30.0     180.0      90.0

[36806400 rows x 7 columns]
Elapsed time: 62.83943759999238


In [19]:
# drop boundary coordinates
df = df.drop(columns=['bnds','lon_bnds','lat_bnds'])
# Only keep one duplicate row
df = df.drop_duplicates()
print(df)

            year    lat     lon  max_dry_day
0         1950.0 -89.75 -179.75         87.0
2         1950.0 -89.75 -179.25         87.0
4         1950.0 -89.75 -178.75         87.0
6         1950.0 -89.75 -178.25         87.0
8         1950.0 -89.75 -177.75         87.0
...          ...    ...     ...          ...
36806390  2020.0  89.75  177.75         30.0
36806392  2020.0  89.75  178.25         30.0
36806394  2020.0  89.75  178.75         30.0
36806396  2020.0  89.75  179.25         30.0
36806398  2020.0  89.75  179.75         30.0

[18403200 rows x 4 columns]


In [20]:
# save as csv and replace the original file
tic()

df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')


elapsed = toc()
print('Elapsed time:',elapsed)

Elapsed time: 171.23807910000323


In [21]:
# report csv file size
csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

csv size: 613.526 Megabytes


## Repeat the process above for remaining files
`max_wet_day.nc`

In [25]:
tic()


filename="max_wet_day.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-cwd-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 452.9719657000096
csv size: 605.734 Megabytes


`one_day_rainfall.nc`

In [24]:
tic()


filename="one_day_rainfall.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-rx1day-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 461.3584544000041
csv size: 627.368 Megabytes


`five_day_rainfall.nc`

In [26]:
tic()


filename="five_day_rainfall.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-rx5day-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 449.6015490999853
csv size: 631.942 Megabytes


`wet_rainfall.nc`

In [27]:
tic()


filename="wet_rainfall.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-r95ptot-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 516.0723609999986
csv size: 641.541 Megabytes


`frost_day.nc`



In [32]:
tic()


filename="frost_day.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-fd-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 1363.9586674999737
csv size: 665.092 Megabytes


`hi35_day.nc`

In [33]:
tic()


filename="hi35_day.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-hi35-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")


Elapsed time: 1232.7362042000168
csv size: 649.37 Megabytes


`hot_day.nc`


In [34]:
tic()


filename="hot_day.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-hd35-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 1112.8847228999948
csv size: 650.616 Megabytes


`ice_day.nc`

In [35]:
tic()


filename="ice_day.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-id-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 1215.6614080000145
csv size: 663.359 Megabytes


`temper.nc`

In [8]:
tic()


filename="mean_temper.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-tas-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 559.996739299997
csv size: 630.808 Megabytes


`rainfall.nc`

In [9]:
tic()


filename="rainfall.nc"
ds = xr.open_dataset(filename)
df = ds.to_dataframe()
df = df.rename(columns = {'timeseries-pr-annual-mean':filename[:-3]}, inplace = False)
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+') # save as csv
df = pd.read_csv(filename[:-3]+'.csv')
df = df.drop(columns=['bnds','lon_bnds','lat_bnds']) # drop boundary coordinates
df = df.drop_duplicates() # Only keep one duplicate row
df.to_csv(filename[:-3]+'.csv',index='False',mode='w+')

print('Elapsed time:',toc())

csvfile = filename[:-3]+'.csv'
size = get_file_size(csvfile)
convert_bytes(size, "MB")

Elapsed time: 574.1694542999903
csv size: 651.394 Megabytes
